In [ ]:
import tensorflow as tf

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Your machine learning code using TensorFlow

In [ ]:
!nvidia-smi

In [ ]:
!pip install scikit-plot

In [ ]:
# Define paths
zip_file_path = '/content/CK+48.zip'  # Path to the uploaded zip file in Colab
extracted_dir_path = '/content/extracted/'  # Adjust the target directory in Colab

# Unzip the file
!unzip -q "$zip_file_path" -d "$extracted_dir_path"

print("Extraction completed.")

In [ ]:
import os
import cv2
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG19
import matplotlib.pyplot as plt

In [ ]:
# Mapping of emotions to numerical labels
emotion_to_label = {'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happy': 4, 'sadness': 5, 'surprise': 6}
label_to_text = {0: 'anger', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happy', 5: 'sadness', 6: 'surprise'}




INPUT_PATH = "/content/extracted/CK+48/"

# Count the total number of images
total_images = sum(len(os.listdir(os.path.join(INPUT_PATH, dir_))) for dir_ in emotion_to_label.keys())

# Initialize arrays for images and labels
img_arr = np.empty(shape=(total_images, 224, 224, 3))
img_label = np.empty(shape=(total_images))

# Initialize counters for indexing
idx = 0

for emotion, label_value in emotion_to_label.items():
    dir_path = os.path.join(INPUT_PATH, emotion)
    for f in os.listdir(dir_path):
        img_path = os.path.join(dir_path, f)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (224, 224))
        img_arr[idx] = img_resized
        img_label[idx] = label_value
        idx += 1